In [39]:
import pandas as pd

In [40]:
# data_df = pd.read_csv("data/data.csv")
# data_2014 = pd.read_csv("data/data_2014.csv")
data_2014_2024 = pd.read_csv("data/data_2014_2024.csv")
indicator_df = pd.read_csv("data/indicators.csv")
region_df = pd.read_csv("data/regions.csv")
# crime_df = pd.read_csv("data/crime.csv")

In [41]:
# data_df_sample = data_df.sample(n=500000).reset_index()
# data_df_sample.to_csv("data_sample.csv", index=False)

In [42]:
# data_df['date'] = pd.to_datetime(data_df['date'])
# data_2014 = data_df[data_df["date"].dt.year == 2014]
# data_2014.to_csv("data/data_2014.csv", index=False)

In [43]:
# data_df['date'] = pd.to_datetime(data_df['date'])
# data_2014_2024 = data_df[(data_df["date"].dt.year >= 2014) & (data_df["date"].dt.year <= 2024)]
# data_2014_2024.to_csv("data/data_2014_2024.csv", index=False)

In [44]:
data_2014_2024.head()

,indicator_id,region_id,date,value
0,Z1BR,20,2017-11-30,153373.519224
1,Z1BR,26,2022-02-28,382094.085250
2,Z1BR,53,2022-02-28,224555.043957
3,Z1BR,45,2022-04-30,84132.786751
4,Z1BR,62,2022-10-31,216070.285522


In [45]:
# data_2014
# region_df

In [46]:
data_2014_2024.dtypes

indicator_id     object
region_id         int64
date             object
value           float64
dtype: object

In [ ]:
# filter zillow data, looking at specific indicator ids(only home values)
data_2014_2024["date"] = pd.to_datetime(data_2014_2024["date"])
data_2014_2024["year"] = data_2014_2024["date"].dt.year
indicator_filter = ['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z5BR', 'Z2BR', 'Z3BR', 'Z1BR', 'Z4BR']
data_2014_2024 = data_2014_2024[data_2014_2024.indicator_id.isin(indicator_filter)]

In [ ]:
# keep latest house listing for each year
data_2014_2024_clean = data_2014_2024.sort_values(by=["date", "indicator_id", "region_id"], ascending=[False, True, True]).drop_duplicates(subset=["year", "indicator_id", "region_id"], keep="first").reset_index()

In [49]:
data_2014_2024_clean.head()

,index,indicator_id,region_id,date,value
0,5569049,Z1BR,3,2024-12-31,180759.247684
1,5568699,Z1BR,4,2024-12-31,176674.250061
2,5568808,Z1BR,6,2024-12-31,136040.104131
3,5568685,Z1BR,8,2024-12-31,226384.365332
4,5568625,Z1BR,9,2024-12-31,510248.444263


In [50]:
data_2014_2024_clean.tail()

,index,indicator_id,region_id,date,value
68429009,4723992,ZSFH,827641,2014-01-31,103651.673074
68429010,4721222,ZSFH,827642,2014-01-31,165813.474380
68429011,4711927,ZSFH,827643,2014-01-31,147200.023145
68429012,4711232,ZSFH,827644,2014-01-31,134829.222175
68429013,21619470,ZSFH,827645,2014-01-31,136158.249216


In [51]:
merged_df = pd.merge(data_2014_2024_clean, region_df, on="region_id", how="left")

In [52]:
merged_df.groupby("region_id").agg("count")

,index,indicator_id,date,value,region_type,region
region_id,,,,,,
3,1320,1320,1320,1320,1320,1320
4,1320,1320,1320,1320,1320,1320
6,1320,1320,1320,1320,1320,1320
8,1320,1320,1320,1320,1320,1320
9,1320,1320,1320,1320,1320,1320
...,...,...,...,...,...,...
845160,70,70,70,70,70,70
845164,70,70,70,70,70,70
845166,62,62,62,62,62,62


In [53]:
check_merge = region_df[region_df["region_id"].isin(data_2014_2024_clean["region_id"])]

In [54]:
print(len(check_merge["region_type"].unique()))

6


In [55]:
check_merge

,region_id,region_type,region
0,96208,zip,"90706;CA;Los Angeles-Long Beach-Anaheim, CA;Be..."
1,394415,metro,"Bridgeport, CT"
2,394653,metro,"Greenville, SC"
3,394312,metro,"Albuquerque, NM"
4,394357,metro,"Bakersfield, CA"
...,...,...,...
89300,59107,zip,"3293;NH;Lebanon, NH-VT;Woodstock;Grafton County"
89301,94027,zip,"83025;WY;Jackson, WY-ID;Wilson;Teton County"
89302,58463,zip,"1718;MA;Boston-Cambridge-Newton, MA-NH;Acton;M..."
89303,75653,zip,40363;KY;nan;nan;Owen County


In [56]:
check_merge_zip = check_merge[check_merge["region_type"] == "zip"].copy()
check_merge_metro = check_merge[check_merge["region_type"] == "metro"].copy()
#check_merge_state = check_merge[check_merge["region_type"] == "state"].copy()
check_merge_neigh = check_merge[check_merge["region_type"] == "neigh"].copy()
check_merge_city = check_merge[check_merge["region_type"] == "city"].copy()
check_merge_county = check_merge[check_merge["region_type"] == "county"].copy()

## Splice regions to only get the city and state from them

In [57]:
check_merge_zip.loc[:, "city"] = check_merge_zip["region"].str.split(";").str[3]
check_merge_zip.loc[:, "state"] = check_merge_zip["region"].str.split(";").str[1]

In [58]:
check_merge_zip

,region_id,region_type,region,city,state
0,96208,zip,"90706;CA;Los Angeles-Long Beach-Anaheim, CA;Be...",Bellflower,CA
72,95315,zip,"87121;NM;Albuquerque, NM;Albuquerque;Bernalill...",Albuquerque,NM
73,91325,zip,"76244;TX;Dallas-Fort Worth-Arlington, TX;Fort ...",Fort Worth,TX
74,91732,zip,"77083;TX;Houston-The Woodlands-Sugar Land, TX;...",Houston,TX
75,61616,zip,"10002;NY;New York-Newark-Jersey City, NY-NJ-PA...",New York,NY
...,...,...,...,...,...
89300,59107,zip,"3293;NH;Lebanon, NH-VT;Woodstock;Grafton County",Woodstock,NH
89301,94027,zip,"83025;WY;Jackson, WY-ID;Wilson;Teton County",Wilson,WY
89302,58463,zip,"1718;MA;Boston-Cambridge-Newton, MA-NH;Acton;M...",Acton,MA
89303,75653,zip,40363;KY;nan;nan;Owen County,nan,KY


In [59]:
check_merge_metro

,region_id,region_type,region
1,394415,metro,"Bridgeport, CT"
2,394653,metro,"Greenville, SC"
3,394312,metro,"Albuquerque, NM"
4,394357,metro,"Bakersfield, CA"
5,394308,metro,"Albany, NY"
...,...,...,...
89082,753924,metro,"Urban Honolulu, HI"
89083,395169,metro,"Tulsa, OK"
89084,394619,metro,"Fresno, CA"
89085,395238,metro,"Worcester, MA"


In [60]:
check_merge_metro = check_merge_metro.rename(columns={"region": 'city_state'})
check_merge_metro

,region_id,region_type,city_state
1,394415,metro,"Bridgeport, CT"
2,394653,metro,"Greenville, SC"
3,394312,metro,"Albuquerque, NM"
4,394357,metro,"Bakersfield, CA"
5,394308,metro,"Albany, NY"
...,...,...,...
89082,753924,metro,"Urban Honolulu, HI"
89083,395169,metro,"Tulsa, OK"
89084,394619,metro,"Fresno, CA"
89085,395238,metro,"Worcester, MA"


In [61]:
check_merge_neigh.loc[:, "city"] = check_merge_neigh["region"].str.split(";").str[4]
check_merge_neigh.loc[:, "state"] = check_merge_neigh["region"].str.split(";").str[1]
check_merge_neigh

,region_id,region_type,region,city,state
155,274772,neigh,Northeast Dallas; TX; Dallas-Fort Worth-Arling...,Dallas,TX
164,273698,neigh,"Far North; TX; Dallas-Fort Worth-Arlington, TX...",Dallas,TX
183,275473,neigh,Southeast Dallas; TX; Dallas-Fort Worth-Arling...,Dallas,TX
306,196538,neigh,Murray Hill; NY; New York-Newark-Jersey City; ...,New York,NY
958,118208,neigh,South Los Angeles;CA;Los Angeles-Long Beach-An...,Los Angeles County,CA
...,...,...,...,...,...
88516,807945,neigh,"Dellwood Park;TX;Laredo, TX;Laredo;Webb County",Webb County,TX
88517,806920,neigh,Lake Mathews;CA;Riverside-San Bernardino-Ontar...,Riverside County,CA
88518,250248,neigh,"Central Park;OK;Tulsa, OK;Tulsa;Tulsa County",Tulsa County,OK
88519,416777,neigh,Archwood Acres;TN;Nashville-Davidson--Murfrees...,Davidson County,TN


In [62]:
check_merge_city.loc[:, "city"] = check_merge_city["region"].str.split(";").str[0]
check_merge_city.loc[:, "state"] = check_merge_city["region"].str.split(";").str[1]
check_merge_city

,region_id,region_type,region,city,state
163,32503,city,"Lincoln;NE;Lincoln, NE;Lancaster County",Lincoln,NE
343,17384,city,"Chandler;AZ;Phoenix-Mesa-Chandler, AZ;Maricopa...",Chandler,AZ
344,13437,city,Randolph Township; NJ; New York-Newark-Jersey ...,Randolph Township,NJ
363,47945,city,"Tallahassee;FL;Tallahassee, FL;Leon County",Tallahassee,FL
364,12065,city,"Irving;TX;Dallas-Fort Worth-Arlington, TX;Dall...",Irving,TX
...,...,...,...,...,...
88998,4292,city,"Dalzell;IL;Ottawa, IL;Bureau County",Dalzell,IL
88999,11875,city,Herndon;KS;nan;Rawlins County,Herndon,KS
89000,23854,city,Brownell;KS;nan;Ness County,Brownell,KS
89002,22347,city,New Pine Creek;OR;nan;Lake County,New Pine Creek,OR


In [63]:
check_merge_county.loc[:, "city"] = check_merge_county["region"].str.split(";").str[0]
check_merge_county.loc[:, "state"] = check_merge_county["region"].str.split(";").str[1]
check_merge_county

,region_id,region_type,region,city,state
880,2841,county,San Diego County;CA;San Diego-Chula Vista-Carl...,San Diego County,CA
915,1286,county,Orange County;CA;Los Angeles-Long Beach-Anahei...,Orange County,CA
916,445,county,"Clark County;NV;Las Vegas-Henderson-Paradise, NV",Clark County,NV
918,2049,county,"Tarrant County;TX;Dallas-Fort Worth-Arlington, TX",Tarrant County,TX
932,386,county,"Bexar County;TX;San Antonio-New Braunfels, TX",Bexar County,TX
...,...,...,...,...,...
82375,2400,county,Manistee County;MI;nan,Manistee County,MI
82494,2353,county,"Humboldt County;NV;Winnemucca, NV",Humboldt County,NV
82799,1077,county,Hardin County;IA;nan,Hardin County,IA
82951,263,county,Otoe County;NE;nan,Otoe County,NE


In [64]:
check_merge_combine = pd.concat([check_merge_zip, check_merge_metro, check_merge_neigh, check_merge_city, check_merge_county], ignore_index=True)

In [65]:
check_merge_combine.head()

,region_id,region_type,region,city,state,city_state
0,96208,zip,"90706;CA;Los Angeles-Long Beach-Anaheim, CA;Be...",Bellflower,CA,NaN
1,95315,zip,"87121;NM;Albuquerque, NM;Albuquerque;Bernalill...",Albuquerque,NM,NaN
2,91325,zip,"76244;TX;Dallas-Fort Worth-Arlington, TX;Fort ...",Fort Worth,TX,NaN
3,91732,zip,"77083;TX;Houston-The Woodlands-Sugar Land, TX;...",Houston,TX,NaN
4,61616,zip,"10002;NY;New York-Newark-Jersey City, NY-NJ-PA...",New York,NY,NaN


In [66]:
states = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

In [67]:
merged_df_clean = pd.merge(data_2014_2024_clean, check_merge_combine.drop(["region_type","region"], axis=1), on="region_id", how="inner")
merged_df_clean["date"] = pd.to_datetime(merged_df_clean["date"])
merged_df_clean

,index,indicator_id,region_id,date,value,city,state,city_state
0,18904615,Z1BR,66,2024-12-31,343944.668882,Ada County,ID,NaN
1,2744186,Z2BR,66,2024-12-31,404719.864288,Ada County,ID,NaN
2,3229436,Z3BR,66,2024-12-31,489884.487344,Ada County,ID,NaN
3,3735579,Z4BR,66,2024-12-31,607515.931325,Ada County,ID,NaN
4,2453920,Z5BR,66,2024-12-31,775477.535805,Ada County,ID,NaN
...,...,...,...,...,...,...,...,...
68361689,20977255,ZSFH,165340,2014-05-31,190218.000000,Moody,ME,NaN
68361690,20977254,ZSFH,165340,2014-04-30,190249.000000,Moody,ME,NaN
68361691,20977253,ZSFH,165340,2014-03-31,191243.000000,Moody,ME,NaN
68361692,20977252,ZSFH,165340,2014-02-28,192496.000000,Moody,ME,NaN


In [68]:
#crime_df

In [69]:
merged_df_clean["city_state"] = merged_df_clean["city"] + ", " + merged_df_clean["state"]
final_zilow_df = pd.merge(merged_df_clean, indicator_df, on="indicator_id", how="inner")
# old crime dataset stuff that is no longer needed
#crime_df["state_abbrev"] = crime_df["State"].map(states)
#crime_df["city_state"] = crime_df["City"] + ", " + crime_df["state_abbrev"]
#crime_df_max = crime_df.groupby(["city_state", "Year", "Month"])["Incident"].max().reset_index()
#crime_df_sum = crime_df_max.groupby("city_state")["Incident"].sum()
#final_df = pd.merge(final_zilow_df, crime_df_sum.rename("total_incidents"), on="city_state", how="inner")

In [70]:
merged_df_clean.head()

,index,indicator_id,region_id,date,value,city,state,city_state
0,18904615,Z1BR,66,2024-12-31,343944.668882,Ada County,ID,"Ada County, ID"
1,2744186,Z2BR,66,2024-12-31,404719.864288,Ada County,ID,"Ada County, ID"
2,3229436,Z3BR,66,2024-12-31,489884.487344,Ada County,ID,"Ada County, ID"
3,3735579,Z4BR,66,2024-12-31,607515.931325,Ada County,ID,"Ada County, ID"
4,2453920,Z5BR,66,2024-12-31,775477.535805,Ada County,ID,"Ada County, ID"


In [ ]:
final_zilow_df["year_city_state"] = final_zilow_df["date"].dt.year.astype(str) + ": " + final_zilow_df["city_state"]
final_zilow_df

In [ ]:
# no longer needed since we found a better dataset
# final_df.to_csv("data/final_df.csv", index=False)

In [ ]:
better_crime_df = pd.read_csv("data/better_crime.csv")

C:\Users\zacha\AppData\Local\Temp\ipykernel_20772\1849724017.py:1: DtypeWarning: Columns (19,20,25,50) have mixed types. Specify dtype option on import or set low_memory=False.
  better_crime_df = pd.read_csv("data/better_crime.csv")


In [ ]:
better_crime_df.head()

,ori,ori9,agency_name,state,state_abb,year,number_of_months_missing,number_of_months_reported,last_month_reported,fips_state_code,...,unfounded_motor_vehicle_theft_total,unfounded_motor_vehicle_theft_car,unfounded_motor_vehicle_theft_truck,unfounded_motor_vehicle_theft_other,unfounded_all_crimes,unfounded_theft_under50_dollar,unfounded_assault_aggravated,unfounded_index_violent,unfounded_index_property,unfounded_index_total
0,AK00101,AK0010100,anchorage,alaska,AK,2024,NaN,12,december,2.0,...,31,22,7,2,91,0,6,13,67,80
1,AK00101,AK0010100,anchorage,alaska,AK,2023,NaN,12,december,2.0,...,40,33,6,1,144,0,12,22,97,119
2,AK00101,AK0010100,anchorage,alaska,AK,2022,NaN,12,december,2.0,...,52,40,12,0,99,0,2,12,87,99
3,AK00101,AK0010100,anchorage,alaska,AK,2021,NaN,0,no months reported,2.0,...,0,0,0,0,0,0,0,0,0,0
4,AK00101,AK0010100,anchorage,alaska,AK,2020,NaN,12,december,2.0,...,37,22,15,0,89,0,0,5,84,89


In [ ]:
better_crime_df = better_crime_df[better_crime_df["address_city"].str.len() > 2]
better_crime_df["address_city"].value_counts()

address_city
harrisburg       7697
indianapolis     5932
columbia         5875
albany           4517
tallahassee      3943
                 ... 
krugerville        13
felicity           13
walthourville      13
chimney rock       13
loving             13
Name: count, Length: 9034, dtype: int64

In [ ]:
crime_2014_2024 = better_crime_df[(better_crime_df["year"] >= 2014) & (better_crime_df["year"] <= 2024)]
#crime_2014_2024.to_csv("data/better_crime_subset.csv", index=False)

In [ ]:
crime_2014_2024["city_state"] = crime_2014_2024["address_city"].str.capitalize() + ", " + crime_2014_2024["address_state"].str.upper()
crime_2014_2024["year_city_state"] = crime_2014_2024["year"].astype(str) + ": " + crime_2014_2024["city_state"]

C:\Users\zacha\AppData\Local\Temp\ipykernel_20772\3352993290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_2014_2024["city_state"] = crime_2014_2024["address_city"].str.capitalize() + ", " + crime_2014_2024["address_state"].str.upper()
C:\Users\zacha\AppData\Local\Temp\ipykernel_20772\3352993290.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_2014_2024["year_city_state"] = crime_2014_2024["year"].astype(str) + ": " + crime_2014_2024["city_state"]


In [ ]:
crime_2014_2024_1 = crime_2014_2024[["year", "city_state", "year_city_state", "actual_murder", "actual_rape_total", "actual_robbery_total", "actual_assault_total", "actual_theft_total"]].copy()
crime_clean = crime_2014_2024_1.groupby(["year", "city_state", "year_city_state"]).agg("sum").reset_index()
crime_clean

,year,city_state,year_city_state,actual_murder,actual_rape_total,actual_robbery_total,actual_assault_total,actual_theft_total
0,2014,"Abbeville, AL","2014: Abbeville, AL",0,4,2,201,114
1,2014,"Abbeville, GA","2014: Abbeville, GA",1,6,1,48,12
2,2014,"Abbeville, LA","2014: Abbeville, LA",1,2,16,521,342
3,2014,"Abbeville, SC","2014: Abbeville, SC",1,8,4,339,293
4,2014,"Abbotsford, WI","2014: Abbotsford, WI",0,0,0,3,42
...,...,...,...,...,...,...,...,...
144252,2024,"Zionsville, IN","2024: Zionsville, IN",1,2,1,33,78
144253,2024,"Zolfo springs, FL","2024: Zolfo springs, FL",0,0,0,0,0
144254,2024,"Zumbrota, MN","2024: Zumbrota, MN",0,0,0,6,13
144255,2024,"Zuni, NM","2024: Zuni, NM",0,0,0,1,0


,index,indicator_id,region_id,date,value,city,state,city_state,indicator,category,year_city_state
0,5569049,Z1BR,3,2024-12-31,180759.247684,NaN,NaN,NaN,ZHVI 1-Bedroom Time Series ($),Home values,NaN
1,5568621,Z1BR,3,2024-11-30,180264.968769,NaN,NaN,NaN,ZHVI 1-Bedroom Time Series ($),Home values,NaN
2,5568147,Z1BR,3,2024-10-31,180166.640859,NaN,NaN,NaN,ZHVI 1-Bedroom Time Series ($),Home values,NaN
3,5567715,Z1BR,3,2024-09-30,180134.608405,NaN,NaN,NaN,ZHVI 1-Bedroom Time Series ($),Home values,NaN
4,5567242,Z1BR,3,2024-08-31,180651.246874,NaN,NaN,NaN,ZHVI 1-Bedroom Time Series ($),Home values,NaN


In [ ]:
final_df = pd.merge(final_zilow_df, crime_clean, on="year_city_state", how="inner")
# Select and rename columns immediately to reduce memory footprint
final_df = final_df[["date", "year_x", "city_state_x", "indicator_id", "indicator", "value", "actual_murder", "actual_rape_total", "actual_robbery_total", "actual_assault_total", "actual_theft_total"]]
final_df.columns = ["date", "year", "city_state", "indicator_id", "indicator", "value", "actual_murder", "actual_rape_total", "actual_robbery_total", "actual_assault_total", "actual_theft_total"]
# Sort after reducing memory usage
final_df = final_df.sort_values(by=["date"], ascending=[False]).reset_index(drop=True)

MemoryError: Unable to allocate 1.03 GiB for an array with shape (7, 19754709) and data type object

In [ ]:
final_df.shape

,date,year,city_state,indicator_id,indicator,value,actual_murder,actual_rape_total,actual_robbery_total,actual_assault_total,actual_theft_total
0,2024-01-31,2024,"Addison, IL",Z1BR,ZHVI 1-Bedroom Time Series ($),142235.056283,0,17,8,436,189
1,2024-01-31,2024,"Addison, IL",Z1BR,ZHVI 1-Bedroom Time Series ($),138766.637985,0,17,8,436,189
2,2024-01-31,2024,"Addison, IL",Z2BR,ZHVI 2-Bedroom Time Series ($),265374.589678,0,17,8,436,189
3,2024-01-31,2024,"Addison, IL",Z2BR,ZHVI 2-Bedroom Time Series ($),249939.270333,0,17,8,436,189
4,2024-01-31,2024,"Addison, IL",Z3BR,ZHVI 3-Bedroom Time Series ($),351889.468032,0,17,8,436,189
...,...,...,...,...,...,...,...,...,...,...,...
1710962,2024-01-31,2024,"Avoca, WI",ZATT,ZHVI All Homes- Top Tier Time Series ($),453618.990878,0,0,0,0,0
1710964,2024-01-31,2024,"Kadoka, SD",ZATT,ZHVI All Homes- Top Tier Time Series ($),247631.938981,0,0,1,4,9
1710967,2024-01-31,2024,"Douglas, AL",ZATT,ZHVI All Homes- Top Tier Time Series ($),274156.656603,0,0,0,43,9
1710969,2024-01-31,2024,"Farina, IL",ZATT,ZHVI All Homes- Top Tier Time Series ($),155433.783177,0,0,0,2,2


In [ ]:
final_df = final_df.groupby(["date", "year", "city_state", "indicator_id", "indicator"]).agg("mean").reset_index()

In [ ]:
final_df.head(10)

,date,year,city_state,indicator_id,indicator,value,actual_murder,actual_rape_total,actual_robbery_total,actual_assault_total,actual_theft_total
0,2014-01-31,2014,"Abbeville, AL",Z2BR,ZHVI 2-Bedroom Time Series ($),72328.500000,0.0,4.0,2.0,201.0,114.0
1,2014-01-31,2014,"Abbeville, AL",Z3BR,ZHVI 3-Bedroom Time Series ($),114569.470400,0.0,4.0,2.0,201.0,114.0
2,2014-01-31,2014,"Abbeville, AL",Z4BR,ZHVI 4-Bedroom Time Series ($),131264.500000,0.0,4.0,2.0,201.0,114.0
3,2014-01-31,2014,"Abbeville, AL",Z5BR,ZHVI 5+ Bedroom Time Series ($),171610.500000,0.0,4.0,2.0,201.0,114.0
4,2014-01-31,2014,"Abbeville, AL",ZABT,ZHVI All Homes- Bottom Tier Time Series ($),48195.275326,0.0,4.0,2.0,201.0,114.0
5,2014-01-31,2014,"Abbeville, AL",ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",95873.847229,0.0,4.0,2.0,201.0,114.0
6,2014-01-31,2014,"Abbeville, AL",ZATT,ZHVI All Homes- Top Tier Time Series ($),215166.977093,0.0,4.0,2.0,201.0,114.0
7,2014-01-31,2014,"Abbeville, AL",ZSFH,ZHVI Single-Family Homes Time Series ($),95452.234986,0.0,4.0,2.0,201.0,114.0
8,2014-01-31,2014,"Abbeville, GA",Z2BR,ZHVI 2-Bedroom Time Series ($),37738.000000,1.0,6.0,1.0,48.0,12.0
9,2014-01-31,2014,"Abbeville, GA",Z3BR,ZHVI 3-Bedroom Time Series ($),51521.000000,1.0,6.0,1.0,48.0,12.0


In [ ]:
final_df.head(15)

,date,year,city_state,indicator_id,indicator,value,actual_murder,actual_rape_total,actual_robbery_total,actual_assault_total,actual_theft_total
0,2024-01-31,2024,"Addison, IL",Z1BR,ZHVI 1-Bedroom Time Series ($),142235.056283,0,17,8,436,189
1,2024-01-31,2024,"Addison, IL",Z1BR,ZHVI 1-Bedroom Time Series ($),138766.637985,0,17,8,436,189
2,2024-01-31,2024,"Addison, IL",Z2BR,ZHVI 2-Bedroom Time Series ($),265374.589678,0,17,8,436,189
3,2024-01-31,2024,"Addison, IL",Z2BR,ZHVI 2-Bedroom Time Series ($),249939.270333,0,17,8,436,189
4,2024-01-31,2024,"Addison, IL",Z3BR,ZHVI 3-Bedroom Time Series ($),351889.468032,0,17,8,436,189
5,2024-01-31,2024,"Addison, IL",Z3BR,ZHVI 3-Bedroom Time Series ($),332009.724073,0,17,8,436,189
6,2024-01-31,2024,"Addison, IL",Z4BR,ZHVI 4-Bedroom Time Series ($),464157.945150,0,17,8,436,189
7,2024-01-31,2024,"Addison, IL",Z4BR,ZHVI 4-Bedroom Time Series ($),442829.364663,0,17,8,436,189
8,2024-01-31,2024,"Addison, IL",Z5BR,ZHVI 5+ Bedroom Time Series ($),576858.531571,0,17,8,436,189
9,2024-01-31,2024,"Addison, IL",Z5BR,ZHVI 5+ Bedroom Time Series ($),551214.622657,0,17,8,436,189
